In [2]:
# importing all the required Libraries
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, recall_score, cohen_kappa_score, precision_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten, Activation, GlobalAveragePooling2D, Conv2D, MaxPooling2D
import tensorflow
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer, LabelEncoder
from tensorflow.keras.applications.inception_v3 import InceptionV3  # InceptionV3
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2  # MobileNetV2
from tensorflow.keras.applications.densenet import DenseNet121  # DenseNet121
from tensorflow.keras.applications.densenet import DenseNet169  # DenseNet169
from tensorflow.keras.applications.mobilenet import MobileNet  # MobileNet
from tensorflow.keras.applications.xception import Xception  # Xception
from tensorflow.keras.applications import ResNet101  # ResNet 101
from tensorflow.keras.applications.resnet50 import ResNet50  # ResNet50
from tensorflow.keras.applications.vgg19 import VGG19  # VGG19
from tensorflow.keras.applications.vgg16 import VGG16  # VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.metrics import AUC
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier  # XGBClassifier
from sklearn.ensemble import AdaBoostClassifier  # AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier  # RandomForestClassifier
from sklearn.svm import SVC  # SVM
from sklearn.neighbors import KNeighborsClassifier  # KNeighborsClassifier
from sklearn.utils import compute_class_weight
from sklearn.utils import shuffle
from sklearn import metrics
from time import time
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from collections import Counter
import cv2
import glob
import skimage
import numpy as np
import pandas as pd
import seaborn as sn
from tqdm import tqdm
from os import listdir
import matplotlib.pyplot as plt
from skimage.transform import resize
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

sn.set()


In [ ]:
def loadImages(path):
    '''
        parameters
        ----------
        path : input path of the images
        
        returns
        -------
        loadedImages : list of loaded images 
    '''
    from tqdm import tqdm
    import cv2
    sample = []
    for filename in tqdm(glob.glob(path)):

        img = cv2.imread(filename)
        img = skimage.transform.resize(img, (224, 224, 3))
        IMG = np.array(img)
        sample.append(IMG)

    return sample


In [ ]:
train_path_1='/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/Data/BRATS/mod_data1/Working_Data/Case_1/train/MGMT_negative/*.png'
train_path_2 = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/Data/BRATS/mod_data1/Working_Data/Case_1/train/MGMT_positive/*.png'

test_path_1 = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/Data/BRATS/mod_data1/Working_Data/Case_1/test/MGMT_negative/*.png'
test_path_2 = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/Data/BRATS/mod_data1/Working_Data/Case_1/test/MGMT_positive/*.png'

In [ ]:
train_neg = loadImages(train_path_1)
train_pos = loadImages(train_path_2)

100%|██████████| 5368/5368 [00:13<00:00, 384.56it/s]


In [ ]:
test_neg = loadImages(test_path_1)
test_pos = loadImages(test_path_2)

100%|██████████| 1287/1287 [00:03<00:00, 377.25it/s]


In [ ]:
df_train_neg = pd.DataFrame({'images': train_neg, 'labels': 0})
df_train_pos = pd.DataFrame({'images': train_pos, 'labels': 1})

df_test_neg = pd.DataFrame({'images': test_neg, 'labels': 0})
df_test_pos = pd.DataFrame({'images': test_pos, 'labels': 1})

# deleting vriables to ensure memory is not full
del train_neg, train_pos, test_neg, test_pos

In [ ]:
final_data = [df_train_neg, df_train_pos, df_test_neg, df_test_pos]
final_data = pd.concat(final_data)

print(final_data.shape)

# delete datframes to ensure memory is not full
del df_train_neg, df_train_pos, df_test_neg, df_test_pos

(12804, 2)


In [ ]:
train_data = final_data['images']
train_labels = final_data['labels']

In [ ]:
print("Labels Count:",Counter(np.array(train_labels)))

Labels Count: Counter({1: 6655, 0: 6149})


In [ ]:
onehot = LabelEncoder()
train_labels = onehot.fit_transform(train_labels)
print(train_labels)

[0 0 0 ... 1 1 1]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels,
                                                    test_size=0.3,
                                                    stratify=train_labels,
                                                    shuffle=True,
                                                    random_state=42)

print('length X_train:', len(X_train))
print('length y_train:', len(y_train))

print('length X_test:',  len(X_test))
print('length y_test:', len(y_test))


length X_train: 8962
length y_train: 8962
length X_test: 3842
length y_test: 3842


In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)


In [ ]:
x_train = np.empty(
    (len(X_train), X_train[0].shape[0], X_train[0].shape[1], X_train[0].shape[2]))
for i, x in enumerate(X_train):
    x_train[i] = X_train[i]
print(x_train.shape)


(8962, 120, 120, 3)


In [ ]:
x_test = np.empty(
    (len(X_test), X_test[0].shape[0], X_test[0].shape[1], X_test[0].shape[2]))
for i, x in enumerate(X_test):
    x_test[i] = X_test[i]
print(x_test.shape)


(3842, 120, 120, 3)


In [3]:
base_model = VGG19(include_top=True, weights='imagenet', input_shape=(224,224,3))


2022-12-08 17:21:04.956872: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-08 17:21:04.957581: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [4]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
x = base_model.output
x = Dropout(0.5)(x)
x = Flatten()(x)
x = BatchNormalization()(x)
x = Dense(1024, kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

predictions = Dense(4, activation='softmax')(x)

model_feat = Model(inputs=base_model.input, outputs=predictions)

train_features = model_feat.predict(x_train)
test_features = model_feat.predict(x_test)


2022-12-07 15:41:29.092909: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


121/121 [==============================] - 11s 95ms/step


In [ ]:
# total features in the model
print(train_features.shape)
print(test_features.shape)

(8962, 4)
(3842, 4)


In [ ]:
x_train_acc, x_val_acc, y_train_acc, y_val_acc = train_test_split(train_features, y_train,
                                                                  test_size=0.3,
                                                                  stratify=y_train,
                                                                  shuffle=True,
                                                                  random_state=42)


In [ ]:
X_test, y_test = test_features, y_test

print('length X_train:', len(x_train_acc))
print('length y_train:', len(y_train_acc))

print('length X_val:',  len(x_val_acc))
print('length y_val:', len(y_val_acc))

print('length X_test:',  len(X_test))
print('length y_test:', len(y_test))


length X_train: 6273
length y_train: 6273
length X_val: 2689
length y_val: 2689
length X_test: 3842
length y_test: 3842


In [ ]:
def classifier_summary(pipeline, X_train, y_train, X_val, y_val, X_test, y_test):
    sentiment_fit = pipeline.fit(X_train, y_train)

    y_pred_train = sentiment_fit.predict(X_train)
    y_pred_val = sentiment_fit.predict(X_val)
    y_pred_test = sentiment_fit.predict(X_test)

    train_accuracy = np.round(accuracy_score(y_train, y_pred_train), 4)*100
    train_precision = np.round(precision_score(
        y_train, y_pred_train, average='weighted'), 4)
    train_recall = np.round(recall_score(
        y_train, y_pred_train, average='weighted'), 4)
    train_F1 = np.round(f1_score(y_train, y_pred_train, average='weighted'), 4)
    train_kappa = np.round(cohen_kappa_score(y_train, y_pred_train), 4)
    train_confusion_matrix = confusion_matrix(y_train, y_pred_train)

    val_accuracy = np.round(accuracy_score(y_val, y_pred_val), 4)*100
    val_precision = np.round(precision_score(
        y_val, y_pred_val, average='weighted'), 4)
    val_recall = np.round(recall_score(
        y_val, y_pred_val, average='weighted'), 4)
    val_F1 = np.round(f1_score(y_val, y_pred_val, average='weighted'), 4)
    val_kappa = np.round(cohen_kappa_score(y_val, y_pred_val), 4)
    val_confusion_matrix = confusion_matrix(y_val, y_pred_val)

    test_accuracy = np.round(accuracy_score(y_test, y_pred_test), 4)*100
    test_precision = np.round(precision_score(
        y_test, y_pred_test, average='weighted'), 4)
    test_recall = np.round(recall_score(
        y_test, y_pred_test, average='weighted'), 4)
    test_F1 = np.round(f1_score(y_test, y_pred_test, average='weighted'), 4)
    test_kappa = np.round(cohen_kappa_score(y_test, y_pred_test), 4)
    test_confusion_matrix = confusion_matrix(y_test, y_pred_test)

    print()
    print('------------------------ Train Set Metrics------------------------')
    print()
    print("accuracy : {}%".format(train_accuracy))
    print("F1_score : {}".format(train_F1))
    print("Cohen Kappa Score : {} ".format(train_kappa))
    print("Recall : {}".format(train_recall))
    print("Precision : {}".format(train_precision))
    print("Confusion Matrix :\n {}".format(train_confusion_matrix))

    print()
    print('------------------------ Validation Set Metrics------------------------')
    print()
    print("accuracy : {}%".format(val_accuracy))
    print("F1_score : {}".format(val_F1))
    print("Cohen Kappa Score : {} ".format(val_kappa))
    print("Recall : {}".format(val_recall))
    print("Precision : {}".format(val_precision))
    print("Confusion Matrix :\n {}".format(val_confusion_matrix))

    print()
    print('------------------------ Test Set Metrics------------------------')
    print()
    print("accuracy : {}%".format(test_accuracy))
    print("F1_score : {}".format(test_F1))
    print("Cohen Kappa Score : {} ".format(test_kappa))
    print("Recall : {}".format(test_recall))
    print("Precision : {}".format(test_precision))
    print("Confusion Matrix : {}".format(test_confusion_matrix))

    print("-"*80)
    print()


In [ ]:
names = [
        "K Nearest Neighbour Classifier",
        'SVM',
        "Random Forest Classifier",
        "AdaBoost Classifier", 
        "XGB Classifier",
         ]
classifiers = [
    KNeighborsClassifier(),
    SVC(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    XGBClassifier(),
        ]

zipped_clf = zip(names,classifiers)

In [ ]:
def classifier_comparator(X_train,y_train,X_val,y_val,X_test,y_test,classifier=zipped_clf): 
    result = []
    for n,c in classifier:
        checker_pipeline = Pipeline([
            ('classifier', c)
        ])
        print("Fitting {} on features ".format(n))
        #print(c)
        classifier_summary(checker_pipeline,X_train, y_train, X_val, y_val,X_test,y_test)

In [ ]:
classifier_comparator(x_train_acc,y_train_acc,x_val_acc,y_val_acc,X_test,y_test,classifier=zipped_clf)

Fitting K Nearest Neighbour Classifier on features 

------------------------ Train Set Metrics------------------------

accuracy : 71.32%
F1_score : 0.7129
Cohen Kappa Score : 0.4247 
Recall : 0.7132
Precision : 0.7131
Confusion Matrix :
 [[2054  959]
 [ 840 2420]]

------------------------ Validation Set Metrics------------------------

accuracy : 56.120000000000005%
F1_score : 0.5601
Cohen Kappa Score : 0.1186 
Recall : 0.5612
Precision : 0.5603
Confusion Matrix :
 [[656 635]
 [545 853]]

------------------------ Test Set Metrics------------------------

accuracy : 57.11000000000001%
F1_score : 0.5703
Cohen Kappa Score : 0.139 
Recall : 0.5711
Precision : 0.5703
Confusion Matrix : [[ 970  875]
 [ 773 1224]]
--------------------------------------------------------------------------------

Fitting SVM on features 

------------------------ Train Set Metrics------------------------

accuracy : 59.209999999999994%
F1_score : 0.5638
Cohen Kappa Score : 0.1675 
Recall : 0.5921
Precision :